In [10]:
"""
Utility functions below use the following name conventions:
m_coord_foo : coordinates of foo expressed in the 2D-NE format. For example, array[-258.5, -10.4]
g_coord_foo : coordinates of foo expressed in the 2D-grid format. For example, array[5, 10]

A grid is defined as a dictionary:
{
"Name": name of the grid,
"origin": coordinates of the origin expressed in the NED format - m_coord_ref
"offset"(): offset values expressed in number of square grids. 
For example, a 5 x 5 grid can move its origin to the center with Offset = [-2,-2]
"resolution": number of square grids per meter. 
For example, 10 means each square of 1m will be divided into 100 grids
"size": the size of the grid - for example, (20,20),
"data": the grid filled with some values, usually 0 for free space, 1 for occupied space
}

A map is defined as a dictionary:
{
"Name": name of the map
"Origin": this is the center of the map expressed in geodetic format - (lon0, lat0, alt0)
"Unit": must be expressed in meters
"Data": data representing obstacles,
"min_max_east_north":(east_min, east_max, north_min, north_max)
}
"""


'\nUtility functions below use the following name conventions:\nm_coord_foo : coordinates of foo expressed in the 2D-NE format. For example, array[-258.5, -10.4]\ng_coord_foo : coordinates of foo expressed in the 2D-grid format. For example, array[5, 10]\n\nA grid is defined as a dictionary:\n{\n"Name": name of the grid,\n"origin": coordinates of the origin expressed in the NED format - m_coord_ref\n"offset"(): offset values expressed in number of square grids. \nFor example, a 5 x 5 grid can move its origin to the center with Offset = [-2,-2]\n"resolution": number of square grids per meter. \nFor example, 10 means each square of 1m will be divided into 100 grids\n"size": the size of the grid - for example, (20,20),\n"data": the grid filled with some values, usually 0 for free space, 1 for occupied space\n}\n\nA map is defined as a dictionary:\n{\n"Name": name of the map\n"Origin": this is the center of the map expressed in geodetic format - (lon0, lat0, alt0)\n"Unit": must be expresse

In [11]:
import numpy as np

In [12]:
def create_grid_from_map(my_map, drone_altitude, safety_distance, resolution = 1):
    """
    Returns a grid representation of a 2D configuration space
    based on given obstacle data, drone altitude and safety distance
    arguments.
    """
    
    data = my_map["data"]

    # minimum and maximum north coordinates
    north_min = np.floor(np.min(data[:, 0] - data[:, 3]))
    north_max = np.ceil(np.max(data[:, 0] + data[:, 3]))

    # minimum and maximum east coordinates
    east_min = np.floor(np.min(data[:, 1] - data[:, 4]))
    east_max = np.ceil(np.max(data[:, 1] + data[:, 4]))

    # given the minimum and maximum coordinates we can
    # calculate the size of the grid.
    north_size = int(np.ceil(north_max - north_min) / resolution)
    east_size = int(np.ceil(east_max - east_min) / resolution)

    # Initialize an empty grid
    grid_size = (north_size, east_size)
    grid = np.zeros(grid_size)

    # Populate the grid with obstacles
    for i in range(data.shape[0]):
        north, east, alt, d_north, d_east, d_alt = data[i, :]
        if alt + d_alt + safety_distance > drone_altitude:
            obstacle = [
                int(np.clip((north - d_north - safety_distance - north_min) / resolution, 0, north_size-1)),
                int(np.clip((north + d_north + safety_distance - north_min) / resolution, 0, north_size-1)),
                int(np.clip((east - d_east - safety_distance - east_min) / resolution, 0, east_size-1)),
                int(np.clip((east + d_east + safety_distance - east_min) / resolution, 0, east_size-1)),
            ]
            grid[obstacle[0]:obstacle[1]+1, obstacle[2]:obstacle[3]+1] = 1

    return grid, int(north_min), int(east_min), grid_size

In [13]:
def map_to_grid(m_coord_pt, grid):
    """
    Converts map coordinates to a grid coordinate
    """
    g_coord = [int((m_coord_pt[0] - grid["origin"][0]) / grid["resolution"]),
                 int((m_coord_pt[1] - grid["origin"][1]) / grid["resolution"])]
    return g_coord

def grid_to_map(g_coord, grid):
    """
    Converts grid coordinates to a map coordinate
    """
    m_coord = [g_coord[0] * grid["resolution"] + grid["origin"][0],
                 g_coord[1] * grid["resolution"] + grid["origin"][1]]
    return m_coord

def grid_to_grid(g_coord, grid_a, grid_b):
    """
    Converts a grid A coordinate to a grid B coordinate
    """
    m_coord = grid_to_map(g_coord, grid_a)
    return map_to_grid(m_coord, grid_b)

3D Utility Functions

m_coord_foo : coordinates of foo expressed in the 3D-NED format. For example, array[-258.5, -10.4, -5.0]
g_coord_foo : coordinates of foo expressed in the 3D-grid format. For example, array[5, 10, -5.0]



In [14]:
def map_to_grid_3D(m_coord_pt, grid):
    """
    Converts map coordinates to a grid coordinate
    """
    g_coord = [int((m_coord_pt[0] - grid["origin"][0]) / grid["resolution"]),
               int((m_coord_pt[1] - grid["origin"][1]) / grid["resolution"]),
               int((m_coord_pt[2] - grid["origin"][2]) / grid["resolution"])]
    return g_coord

def grid_to_map_3D(g_coord, grid):
    """
    Converts grid coordinates to a map coordinate
    """
    m_coord = [g_coord[0] * grid["resolution"] + grid["origin"][0],
               g_coord[1] * grid["resolution"] + grid["origin"][1],
               g_coord[2] * grid["resolution"] + grid["origin"][2]]
    return m_coord

def grid_to_grid_3D(g_coord, grid_a, grid_b):
    """
    Converts a 3D_grid A coordinate to a 3D_grid B coordinate
    """
    m_coord = grid_to_map_3D(g_coord, grid_a)
    return map_to_grid_3D(m_coord, grid_b)

In [15]:
def create_moving_grid(current_loc, name, grid_size, res):
    """
    Creates a grid with the current location as the center of the grid
    """
    
    grid = {
    "name": name,
    "origin": np.array(current_loc[:2]) - np.array(grid_size) / 2,
    "offset": (0,0),
    "resolution": res,
    "size": grid_size,
    "data": np.zeros(grid_size)   
    }
    return grid

In [16]:
def in_grid(m_coord_pt, grid):
    """
    Returns True if a point with map coordinates m_coord_pt is in grid
    """
    g_coord_pt = map_to_grid(m_coord_pt, grid)
    test = (0<=g_coord_pt[0]<grid["size"][0]) and (0<=g_coord_pt[1]<grid["size"][1])
    return test

In [230]:
def in_grid_3D(m_coord_pt, grid):
    """
    Returns True if a point with map coordinates m_coord_pt is in grid
    """
    g_coord_pt = map_to_grid_3D(m_coord_pt, grid)
    for i in range(len(m_coord_pt)):
        if not (0<=g_coord_pt[i]<grid["size"][i]):
            return False
    return True

In [231]:
def intersect_grid_pts (pt_a, pt_b, grid):
    """
    Finds the point at the intersection of the line formed by 2 points and the grid
    Assumes we move from pt_a to pt_b
    """
    
    # Notation: y = mx + b - linear Equation
    if abs((pt_b[1] - pt_a[1])) > 0.1:
        m = (pt_b[0] - pt_a[0])/(pt_b[1] - pt_a[1])
        
        if m == 0:
            b = pt_a[0]
        else:
            b = (pt_b[1] * pt_a[0] - pt_a[1] * pt_b[0]) / (pt_b[1] - pt_a[1])
            
            if pt_b[0] > pt_a[0]:
                y1 = grid["origin"][0] + \
                grid["size"][0] * grid["resolution"] - \
                grid["resolution"] /2
                x1 = (y1 - b) / m
            else:
                y1 = grid["origin"][0] + \
                grid["resolution"] /2
                x1 = (y1 - b) / m

        if pt_b[1] > pt_a[1]:
            x2 = grid["origin"][1] + \
            grid["size"][1] * grid["resolution"] - \
            grid["resolution"] /2
            y2 = m * x2 + b
        else:
            x2 = grid["origin"][1] + \
            grid["resolution"] /2
            y2 = m * x2 + b
        
        #print ("The 2 possible points are: {}, {}".format([y1, x1], [y2, x2]))
        if in_grid(np.array([y2, x2]), grid):
            inter_pt = np.array([y2, x2])
        else:
            inter_pt = np.array([y1, x1])
    
    else:
        x1 = pt_b[1]
        if pt_b[0] > pt_a[0]:
            y1 = grid["origin"][0] + \
            grid["size"][0] * grid["resolution"] - \
            grid["resolution"] /2
        else:
            y1 = grid["origin"][0] + \
            grid["resolution"] /2
        inter_pt = np.array([y1, x1])
        
    return inter_pt

# Creation of a map and 2 grids

In [232]:
altitude = 5

In [233]:
data = np.loadtxt('colliders.csv', delimiter=',', dtype='Float64', skiprows=2)
# minimum and maximum north coordinates
north_min = np.floor(np.min(data[:, 0] - data[:, 3]))
north_max = np.ceil(np.max(data[:, 0] + data[:, 3]))

# minimum and maximum east coordinates
east_min = np.floor(np.min(data[:, 1] - data[:, 4]))
east_max = np.ceil(np.max(data[:, 1] + data[:, 4]))


In [234]:
my_map = {
    "name": "My NED map",
    "origin": (-122.396591, 37.793405, 5.0),
    "unit": "meter",
    "data": data,
    "min_max_east_north":(east_min, east_max, north_min, north_max)
         }

In [235]:
my_map_data = np.zeros((10,12))
my_map_data[1,2]=1
my_map_data[2,2]=1
my_map_data[3,2]=1
my_map_data[3,3]=1
my_map_data[4,4]=1

In [236]:
grid, north_offset, east_offset, grid_size = create_grid_from_map (my_map, 5, 5, 1)

In [237]:
my_grid = {
    "name": "Grid from map",
    "origin": (0, 0, 0),
    "offset": (0,0),
    "resolution": 1,
    "size": grid_size,
    "data": grid    
}

In [238]:
another_moving_grid = create_moving_grid(current_loc = (12.5,22.5), 
                                         name = "Another moving grid", 
                                         grid_size = (5,5), 
                                         res = 1)

In [239]:
my_moving_grid = {
    "name": "My Moving Grid",
    "origin": (10,20),
    "offset": (0,0),
    "resolution": 1,
    "size": (5,5),
    "data": np.zeros((5,5))    
}

In [240]:
my_moving_grid

{'name': 'My Moving Grid',
 'origin': (10, 20),
 'offset': (0, 0),
 'resolution': 1,
 'size': (5, 5),
 'data': array([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])}

In [241]:
another_moving_grid

{'name': 'Another moving grid',
 'origin': array([10., 20.]),
 'offset': (0, 0),
 'resolution': 1,
 'size': (5, 5),
 'data': array([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])}

## Test map_to_grid and grid_to_map

In [242]:
m_coord_pt = [-30, 12]
print ("The point {} is in the grid position: {}".
       format(m_coord_pt, map_to_grid(m_coord_pt, my_grid)))

The point [-30, 12] is in the grid position: [-30, 12]


In [243]:
g_coord_pt = [-30, 12]
my_grid["resolution"] = 2
print ("The point {} back in the map coordinates is: {}".
       format(g_coord_pt, grid_to_map(g_coord_pt, my_grid)))

The point [-30, 12] back in the map coordinates is: [-60, 24]


## Test grid_to_moving_grid

In [244]:
g_coord_pt = [0, 0]
my_moving_grid["origin"] = (-250, 15)
my_moving_grid["resolution"] = 1

print ("The point in my_grid {} in the moving_grid coordinates is: {}".
       format(g_coord_pt, grid_to_grid(g_coord_pt, my_grid, my_moving_grid)))

The point in my_grid [0, 0] in the moving_grid coordinates is: [250, -15]


## Test map_to_moving_grid

In [245]:
m_coord_pt = [-30, 12]
my_moving_grid["origin"] = (-32, 10)
my_moving_grid["resolution"] = 1
print ("The point in map {} in the moving_grid coordinates is: {}".
       format(m_coord_pt, map_to_grid(m_coord_pt, my_moving_grid)))

The point in map [-30, 12] in the moving_grid coordinates is: [2, 2]


## Test in_grid

In [246]:
my_moving_grid

{'name': 'My Moving Grid',
 'origin': (-32, 10),
 'offset': (0, 0),
 'resolution': 1,
 'size': (5, 5),
 'data': array([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])}

In [247]:
m_coord_pt = [-28, 14]
print (in_grid(m_coord_pt, my_moving_grid))
map_to_grid(m_coord_pt, my_moving_grid)

True


[4, 4]

In [248]:
my_moving_3D_grid = {
    "name": "My Moving Grid",
    "origin": (10,20,5),
    "offset": (0,0,0),
    "resolution": 1,
    "size": (5,5,5),
    "data": np.zeros((5,5))    
}

In [249]:
m_coord_pt = [14, 24, 9]
print (in_grid_3D(m_coord_pt, my_moving_3D_grid))


True


In [250]:
current_loc = [-29.5, 12.5]
g_coord_current_loc = map_to_grid(current_loc, my_moving_grid)
wp1 = [-28,13]
wp2 = [-27,20]
wp3 = [-26,13.5]
all_wp = [wp1, wp2, wp3]

In [251]:
for wp in all_wp:
    if not in_grid(wp, my_moving_grid):
        out_wp = wp
        break
    else:
        last_wp = wp

In [252]:
(last_wp, out_wp)

([-28, 13], [-27, 20])

In [253]:
goal = intersect_grid_pts(last_wp, out_wp, my_moving_grid)
print (goal)

[-27.78571429  14.5       ]


## Test Intersect_grid_pts

In [254]:
my_test_grid = {
    "name": "My Test Grid",
    "origin": (100,200),
    "offset": (0,0),
    "resolution": 1,
    "size": (10,10),
    "data": np.zeros((10,10))    
}
pt_a = (105, 205) #this is the center of the grid

# test Left then Top
print ("LEFT")
pt_b = (107, 198)
print (intersect_grid_pts(pt_a, pt_b, my_test_grid))

# test Right then Top
print ("RIGHT")
pt_b = (107, 212)
print (intersect_grid_pts(pt_a, pt_b, my_test_grid) - pt_a)

# test Top then Left
print ("TOP")
pt_b = (115,201)
print (intersect_grid_pts(pt_a, pt_b, my_test_grid) - pt_a)

# test Top then Right
print ("TOP")
pt_b = (115,209)
print (intersect_grid_pts(pt_a, pt_b, my_test_grid)-pt_a)


# test Bottom then Left
print ("Bottom")
pt_b = (95, 201)
print (intersect_grid_pts(pt_a, pt_b, my_test_grid)-pt_a)


# test Bottom then Right
print ("Bottom")
pt_b = (95, 209)
print (intersect_grid_pts(pt_a, pt_b, my_test_grid)-pt_a)

# test Right then Bottom
print ("RIGHT")
pt_b = (103, 212)
print (intersect_grid_pts(pt_a, pt_b, my_test_grid) - pt_a)

# test Left then Bottom
print ("Left")
pt_b = (103, 198)
print (intersect_grid_pts(pt_a, pt_b, my_test_grid)-pt_a)

# test bottom only
print ("Bottom only")
pt_b = (50, 205)
print (intersect_grid_pts(pt_a, pt_b, my_test_grid))

# test top only
print ("Top only")
pt_b = (200, 205)
print (intersect_grid_pts(pt_a, pt_b, my_test_grid))

# test right only
print ("Right only")
pt_b = (105, 300)
print (intersect_grid_pts(pt_a, pt_b, my_test_grid))

# test left only
print ("Left only")
pt_b = (105, 100)
print (intersect_grid_pts(pt_a, pt_b, my_test_grid))

LEFT
[106.28571429 200.5       ]
RIGHT
[1.28571429 4.5       ]
TOP
[ 4.5 -1.8]
TOP
[4.5 1.8]
Bottom
[-4.5 -1.8]
Bottom
[-4.5  1.8]
RIGHT
[-1.28571429  4.5       ]
Left
[-1.28571429 -4.5       ]
Bottom only
[100.5 205. ]
Top only
[109.5 205. ]
Right only
[105.  209.5]
Left only
[105.  200.5]


## Use of shapely library and comparison

In [255]:
from shapely.geometry import Polygon, Point, LineString

In [256]:
def test_intersect1():
    pt_a = (105, 205) #this is the center of the grid
    pt_b = (107, 198)
    return intersect_grid_pts(pt_a, pt_b, my_test_grid)
test_intersect1()

array([106.28571429, 200.5       ])

In [257]:
def test_intersect2():
    pt_a = (105, 205)
    pt_b = (107, 198)
    line = LineString ([pt_a, pt_b])
    grid_perimeter = LineString([(100,200),(110,200),(110,210),(100,210)])
    return line.intersection(grid_perimeter).coords[0]
test_intersect2()

(106.42857142857143, 200.0)

In [258]:
# Test Speed

In [259]:
import time
iterations = 10000
t0 = time.time()
for i in range(iterations):
    #method 1
    test_intersect1()
print ("test 1 took {}".format(time.time() - t0))

t0 = time.time()
for i in range (iterations):
    #method 2
    test_intersect2()
print ("test 2 took {}".format(time.time() - t0))

test 1 took 0.07954978942871094
test 2 took 1.1834688186645508


In [260]:
def fill_with_obstacle(current_loc, res_w = res_w, res_h=res_w, my_map=my_map):
    
    moving_square = np.zeros((res_h, res_w))
    for row in range(res_h):
        for column in range(res_w):
            coord_to_test = [current_loc[0] + row - 2, 
                             current_loc[1] + column - 2]
            if coord_to_test[0]<0 or coord_to_test[1]<0:
                moving_square[row][column] = 1
            else:
                if my_map[coord_to_test[0]][coord_to_test[1]] == 1:
                    moving_square[row][column] = 1
    return moving_square


NameError: name 'res_w' is not defined

In [ ]:
def find_horizon(current_loc, all_wp=all_wp, goal=goal, res_w = res_w, res_h=res_w, my_map=my_map):
    # find the horizon point within a given resolution
    
    # build the moving square
    moving_square = fill_with_obstacle(current_loc, res_w, res_h, my_map)
    print (moving_square)
    
    #Is there a waypoint in the moving square?

    return [0,0]
    #return horizon

In [261]:
map_coord = [50, -100]
grid_resolution = 6
print ("Grid resolution : {}".format(grid_resolution))

print ("Initial map coordinates: {}".format(map_coord))
grid_coord = map_to_grid (map_coord, -300, -500, grid_resolution)
print ("Grid coordinates: {}".format(grid_coord))
print ("Verification: {} = {}".
       format(map_coord, grid_to_map(grid_coord, -300, -500, grid_resolution)))

Grid resolution : 6
Initial map coordinates: [50, -100]


TypeError: map_to_grid() takes 2 positional arguments but 4 were given

In [262]:
grid_coord = [10,20]
grid_resolution = 1
moving_grid_resolution = 0.2
moving_grid_center = [-280, -470]
grid_to_moving_grid(grid_coord, moving_grid_center, grid_resolution, moving_grid_resolution)

NameError: name 'grid_to_moving_grid' is not defined

In [263]:
map_coord = [-200, 300]
moving_grid_center = [-202, 301]
moving_grid_resolution = 0.2
map_to_moving_grid(map_coord, moving_grid_center, moving_grid_resolution)

NameError: name 'map_to_moving_grid' is not defined

In [264]:
a = np.array([1, 2])
b = np.array([1.2, 5])
c = a - b

In [265]:
c.astype(int)

array([ 0, -3])

In [266]:
import numpy as np

# Assume all actions cost the same.
class Action:
    """
    An action is represented by a 3 element tuple.

    The first 2 values are the delta of the action relative
    to the current grid position. The third and final value
    is the cost of performing the action.
    NWU = (1,1,1)
    """
    
    def __init__(self):
        self.list_actions = self.build_actions()
        
    def build_actions(self):
        #list_actions = [[[0,0,0], 1], ...]
        list_actions=[]
        arr = np.zeros([3,3,3])
        for idx, _ in np.ndenumerate(arr):
            if not np.array_equal(idx,[1,1,1]):               
                coord = np.array(idx) - [1,1,1]
                cost = np.linalg.norm(np.array(coord) - np.array([0,0,0])).astype(float)
                list_actions.append([coord, cost])        
        return list_actions

In [298]:
def valid_actions_3D(grid, current_node):
    """
    Returns a list of valid actions given a grid and current node.
    """
    if not in_grid_3D(current_node, grid):
        return []
    all_actions = Action()
    valid_actions = all_actions.list_actions
    node_valid_actions = []
    #print ("valid actions are {}".format(valid_actions))
    
    #n, m = grid.shape[0] - 1, grid.shape[1] - 1
    x, y, z = current_node

    # check if the node is off the grid or
    # it's an obstacle
    
    for each_action in valid_actions:
        new_coord = [x + each_action[0][0],
                     y + each_action[0][1], 
                     z + each_action[0][2]]
        
        test_in_grid = in_grid_3D(new_coord, grid)
        #print ("Test in Grid = {} for new coordinates {}".
               #format(test_in_grid, new_coord))
        if test_in_grid:
            test_occupied = (grid["data"][x + each_action[0][0], 
                              y + each_action[0][1], 
                              z + each_action[0][2]] == 1)
            if not test_occupied:
                node_valid_actions.append(each_action)
                
    return node_valid_actions

"""
def a_star_grid(grid, h, start, goal):

    path = []
    path_cost = 0
    queue = PriorityQueue()
    queue.put((0, start))
    visited = set(start)

    branch = {}
    found = False
    i = 0
    while not queue.empty():
        i+=1
        print ("Iterations {}".format(i))
        item = queue.get()
        current_node = item[1]
        if current_node == start:
            current_cost = 0.0
        else:
            current_cost = branch[current_node][0]

        if current_node == goal:
            print('Found a path.')
            found = True
            break
        else:
            for action in valid_actions(grid, current_node):
                # get the tuple representation
                da = action.delta
                next_node = (current_node[0] + da[0], current_node[1] + da[1])
                branch_cost = current_cost + action.cost
                queue_cost = branch_cost + h(next_node, goal)

                if next_node not in visited:
                    visited.add(next_node)
                    branch[next_node] = (branch_cost, current_node, action)
                    queue.put((queue_cost, next_node))

    if found:
        # retrace steps
        n = goal
        path_cost = branch[n][0]
        path.append(goal)
        while branch[n][1] != start:
            path.append(branch[n][1])
            n = branch[n][1]
        path.append(branch[n][1])
    else:
        print('**********************')
        print('Failed to find a path!')
        print('**********************')
    return path[::-1], path_cost
"""


def heuristic(position, goal_position):
    return np.linalg.norm(np.array(position) - np.array(goal_position))

## Test Valid_actions_3D

In [299]:
grid= np.zeros([10,10,10])

In [300]:
my_grid = {
    "name": "Grid from map",
    "origin": (0, 0, 0),
    "offset": (0,0),
    "resolution": 1,
    "size": (10, 10, 10),
    "data": grid    
}

In [312]:
node = [8,8,1]

In [313]:
my_valid_actions_3D = valid_actions_3D(my_grid, node)
print (len(my_valid_actions_3D))

26


In [303]:
my_valid_actions_3D

[]

In [25]:
all_wp = [(1,2), (3,5), (4,7)]
current_location = (40,50)
all_wp.insert(0,current_location)

In [26]:
all_wp

[(40, 50), (1, 2), (3, 5), (4, 7)]

In [27]:
a

[(40, 50), [(1, 2), (3, 5), (4, 7)]]

In [37]:
def create_2_dot_5_grid(data, safety_distance):
    """
    Returns a grid representation of a 2.5D configuration space
    based on given obstacle data, drone altitude and safety distance
    arguments.
    """

    # minimum and maximum north coordinates
    north_min = np.floor(np.min(data[:, 0] - data[:, 3]))
    north_max = np.ceil(np.max(data[:, 0] + data[:, 3]))

    # minimum and maximum east coordinates
    east_min = np.floor(np.min(data[:, 1] - data[:, 4]))
    east_max = np.ceil(np.max(data[:, 1] + data[:, 4]))

    # given the minimum and maximum coordinates we can
    # calculate the size of the grid.
    north_size = int(np.ceil(north_max - north_min))
    east_size = int(np.ceil(east_max - east_min))

    # Initialize an empty grid
    grid_size = (north_size, east_size)
    grid = np.zeros(grid_size)

    # Populate the grid with obstacles
    for i in range(data.shape[0]):
        north, east, alt, d_north, d_east, d_alt = data[i, :]
        
        obstacle = [
            int(np.clip(north - d_north - safety_distance - north_min, 0, north_size-1)),
            int(np.clip(north + d_north + safety_distance - north_min, 0, north_size-1)),
            int(np.clip(east - d_east - safety_distance - east_min, 0, east_size-1)),
            int(np.clip(east + d_east + safety_distance - east_min, 0, east_size-1)),
        ]
        grid[obstacle[0]:obstacle[1]+1, obstacle[2]:obstacle[3]+1] = alt + d_alt + safety_distance

    return grid, int(north_min), int(east_min), grid_size

In [38]:
import numpy as np
SAFETY_DISTANCE = 5
data = np.loadtxt('colliders.csv', delimiter=',', dtype='float32', skiprows=2)
dot_5_grid, _, _, _ = create_2_dot_5_grid(data, SAFETY_DISTANCE)

In [40]:
dot_5_grid[-275+316, -83+446]

165.0